# <center>Approach3: CNN - GRU Network</center>

## Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

## Starter

In [ ]:
import pandas as pd 
import os

In [3]:
df = pd.read_csv('../data/filtered/filtered_df.csv') 

In [4]:
# OPTION1: Convert FEN to grayscale images
from src.feature.cnn.grayscale import fen_to_image
_ = fen_to_image(df["FEN"].iloc[0], save=True)

FileNotFoundError: [Errno 2] No such file or directory: '/teamspace/studios/this_studio/results/cnn/grayscale.png'

In [ ]:
# OPTION2: Convert FEN to rgb images
from src.feature.cnn.rgb import fen_to_rgb_image
_ = fen_to_rgb_image(df["FEN"].iloc[0], save=True)

Chessboard RGB image saved as ../results/cnn/rgb.png


In [ ]:
# OPTION3: One-Hot encode FEN features
from src.feature.cnn.one_hot import fen_to_one_hot
_ = fen_to_one_hot(df["FEN"].iloc[0], add_empty_channel=True, save=True, verbose=True)

One-hot encoding shape: (8, 8, 13)
Chessboard one-hot encoded image saved as ../results/cnn/one_hot.png


In [7]:
# OPTION1: Encode with char
from src.feature.cnn.move_encoders import encode_moves_with_char
encoded_moves = encode_moves_with_char(df["Moves"].iloc[0], max_len=4, verbose=True)


Encoded Moves (Character-based):
 tensor([[ 4, 13,  4, 14],
        [18,  9, 18,  8],
        [18, 10,  2,  8],
        [18,  8,  2,  8],
        [ 7, 13,  2,  8]])


In [8]:
# OPTION2: Encode with square
from src.feature.cnn.move_encoders import encode_moves_64_squares
max_move_len = df['Moves'].apply(lambda x: len(x.split())).max()
print(f"Maximum number of moves: {max_move_len}")
encoded_moves = encode_moves_64_squares(df["Moves"].iloc[0], max_len=max_move_len*2, verbose=True)

Maximum number of moves: 27
Original Moves: e6e7 b2b1 b3c1 b1c1 h6c1
Encoded (start, end, promotion): [(20, 12), (49, 57), (41, 58), (57, 58), (23, 58)]
Padded Sequence: [20, 12, 49, 57, 41, 58, 57, 58, 23, 58, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [5]:
from src.lib.models.nn_learner import ChessPuzzleRatingModel, NNLearner
from src.lib.train_helper import stratified_split
from src.lib.nn_dataset import ChessPuzzleDataset
import torch
X_train, y_train, X_val, y_val, X_test, y_test = stratified_split(df)

model = ChessPuzzleRatingModel()
pretrained_path = "/teamspace/studios/this_studio/results/models/cnn/best_model_124436.0000"
if os.path.exists(pretrained_path):
    print(f"Loading pretrained model from {pretrained_path}")
    model.load_state_dict(torch.load(pretrained_path))
    print("Pretrained model loaded successfully.")

learner_nn = NNLearner(model=model, dataset_class=ChessPuzzleDataset)

learner_nn.train(X_train, y_train, X_val, y_val)
learner_nn.save_model("../results/models/cnn/last_model4.pt")

Loading pretrained model from /teamspace/studios/this_studio/results/models/cnn/best_model_124436.0000
Pretrained model loaded successfully.


Epoch: 1:   0%|          | 0/4176 [00:00<?, ?it/s]/teamspace/studios/this_studio/notebooks/../src/lib/nn_dataset.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(fen_one_hot, dtype=torch.float32), torch.tensor(encoded_moves, dtype=torch.long), torch.tensor(rating, dtype=torch.float32)
100%|██████████| 522/522 [01:21<00:00,  6.37it/s]


Epoch 1/20 - Train Loss: 123773.7833, Val Loss: 142322.0239
Best model saved with loss: 142322.0239


100%|██████████| 522/522 [01:21<00:00,  6.44it/s]


Epoch 2/20 - Train Loss: 123677.4336, Val Loss: 142416.4755


...

Epoch 12/20 - Train Loss: 123295.2500, Val Loss: 142671.4847


100%|██████████| 522/522 [01:19<00:00,  6.54it/s]


Epoch 13/20 - Train Loss: 123263.5207, Val Loss: 142709.7234


Epoch: 14:  11%|█         | 443/4176 [00:35<04:58, 12.51it/s]


KeyboardInterrupt: 

In [6]:
from torch.utils.data import DataLoader, TensorDataset

X_test["Rating"] = y_test.values
test_dataset = ChessPuzzleDataset(X_test, add_empty_channel=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=True)

learner_nn._evaluate(test_loader)

  0%|          | 0/653 [00:00<?, ?it/s]

/teamspace/studios/this_studio/notebooks/../src/lib/nn_dataset.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(fen_one_hot, dtype=torch.float32), torch.tensor(encoded_moves, dtype=torch.long), torch.tensor(rating, dtype=torch.float32)
100%|██████████| 653/653 [01:42<00:00,  6.40it/s]


142457.16685011485

In [10]:
from src.lib.models.nn_learner import ChessPuzzleRatingModel, NNLearner
from src.lib.train_helper import stratified_split
from src.lib.nn_dataset import ChessPuzzleDataset
import torch
X_train, y_train, X_val, y_val, X_test, y_test = stratified_split(df)

model1 = ChessPuzzleRatingModel()
pretrained_path = "/teamspace/studios/this_studio/results/models/cnn/best_model_142635"
if os.path.exists(pretrained_path):
    print(f"Loading pretrained model from {pretrained_path}")
    model1.load_state_dict(torch.load(pretrained_path))
    print("Pretrained model loaded successfully.")

learner_nn1 = NNLearner(model1, dataset_class=ChessPuzzleDataset)
learner_nn1._evaluate(test_loader)

Loading pretrained model from /teamspace/studios/this_studio/results/models/cnn/best_model_142635
Pretrained model loaded successfully.


  0%|          | 0/653 [00:00<?, ?it/s]/teamspace/studios/this_studio/notebooks/../src/lib/nn_dataset.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(fen_one_hot, dtype=torch.float32), torch.tensor(encoded_moves, dtype=torch.long), torch.tensor(rating, dtype=torch.float32)
100%|██████████| 653/653 [01:42<00:00,  6.36it/s]


142627.762705781